In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import os
import json
import librosa
import pickle
from itertools import cycle

# Set the theme for seaborn plots
sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

# Mount Google Drive to access files (if you're using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Change the working directory to the thesis folder in Google Drive
thesis_path = #path
os.chdir(thesis_path)

# Verify the current working directory
print("Current working directory:", os.getcwd())

# Function to correct audio paths in the JSON data
def correct_paths(data, base_path):
    corrected_data = []
    for item in data:
        if 'audio' in item:
            # Replace backslashes with slashes for compatibility
            corrected_audio_path = item['audio'].replace('\\', '/')
            # Ensure the path starts with the base path
            if not corrected_audio_path.startswith(base_path):
                corrected_audio_path = os.path.join(base_path, corrected_audio_path)
            # Update the item's audio path
            item['audio'] = corrected_audio_path
        corrected_data.append(item)
    return corrected_data

# Function to process datasets
def process_dataset(json_path, base_path, task_filter, dataset_name):
    with open(json_path) as file:
        data = json.load(file)
    print(f"\nProcessing {dataset_name} dataset...")
    print(f"Total items in the {dataset_name} dataset: {len(data)}")

    # Debug: Check the keys in the first few entries
    print(f"Keys in the first item: {data[0].keys()}")

    # Correct paths
    data = correct_paths(data, base_path)

    # Filter by task
    filtered_data = [item for item in data if item['task'] in task_filter]
    print(f"Number of items found with tasks {task_filter}: {len(filtered_data)}")

    # Debug: Check the first few filtered items
    for i, item in enumerate(filtered_data[:5]):
        print(f"Filtered item {i+1}: {item}")

    # Preprocess audio files
    audio_lengths_before = []
    audio_lengths_after = []
    for idx, item in enumerate(filtered_data):
        audio_path = item['audio']
        try:
            audio, sr = librosa.load(audio_path, sr=44100)
            length_before = len(audio) / sr
            audio_lengths_before.append(length_before)

            # Trim silence
            audio_trimmed, _ = librosa.effects.trim(audio, top_db=20)
            length_after = len(audio_trimmed) / sr
            audio_lengths_after.append(length_after)

            # Update the dataset with trimmed audio
            item['audio'] = audio_trimmed

            # Debug every 10 files
            if (idx + 1) % 10 == 0:
                print(f"Processed {idx + 1}/{len(filtered_data)} files from {dataset_name}")
        except Exception as e:
            print(f"Error processing file {audio_path}: {e}")

    print(f"Number of audio files successfully processed in {dataset_name}: {len(audio_lengths_after)}")
    return filtered_data, audio_lengths_before, audio_lengths_after

# Process each dataset
newdata = []

pcgita_data, pcgita_lengths_before, pcgita_lengths_after = process_dataset(
    'PCGITA.json', 'DATI', ['vowel','vowels'], 'PC-GITA'
)
newdata.extend(pcgita_data)

BARI_data, BARI_lengths_before, BARI_lengths_after = process_dataset(
    'BARI_new.json', 'DATI', ['vowels', 'vowelA'], 'BARI'
)
newdata.extend(BARI_data)

molinette_data, molinette_lengths_before, molinette_lengths_after = process_dataset(
    'MOLINETTE_combined.json', 'DATI', ['vowel', 'vowels'], 'MOLINETTE'
)
newdata.extend(molinette_data)

# Debug: Check final dataset size and first few items
print(f"\nTotal size of the combined dataset: {len(newdata)}")
print("First few items in the combined dataset:")
for i, item in enumerate(newdata[:5]):
    print(f"Item {i+1}: {item}")

# Save combined dataset to pickle file
file_path_updated = '/content/drive/MyDrive/thesis/newdata_updated.pkl'
with open(file_path_updated, 'wb') as f:
    pickle.dump(newdata, f)

print(f"\nCombined dataset successfully saved to {file_path_updated}")




In [ ]:
# Plot histograms for each dataset
datasets = [
    ('PC-GITA', pcgita_lengths_before, pcgita_lengths_after),
    ('BARI', BARI_lengths_before, BARI_lengths_after),
    ('MOLINETTE', molinette_lengths_before, molinette_lengths_after)
]

for name, lengths_before, lengths_after in datasets:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(lengths_before, bins=30, alpha=0.7, color='blue')
    plt.xlabel('Audio Length (seconds)')
    plt.ylabel('Number of Audio Files')
    plt.title(f'Distribution of Audio Lengths Before Trimming ({name})')

    plt.subplot(1, 2, 2)
    plt.hist(lengths_after, bins=30, alpha=0.7, color='red')
    plt.xlabel('Audio Length (seconds)')
    plt.ylabel('Number of Audio Files')
    plt.title(f'Distribution of Audio Lengths After Trimming ({name})')

    plt.tight_layout()
    plt.show()

In [ ]:
# Path to the .pkl file
file_path = 'path/to/your/data.pkl'

# Load the .pkl file
with open(file_path, 'rb') as f:
    dataset = pickle.load(f)

# Total number of elements in the dataset
print(f"Total number of elements in dataset: {len(dataset)}")

# Inspect keys available in the first element
print("\nKeys in the first element:")
print(dataset[0].keys())

# Display the first 5 elements
print("\nFirst 5 elements in dataset:")
for i, item in enumerate(dataset[:5]):
    print(f"Element {i + 1}: {item}")

# Statistics for some keys
if 'audio' in dataset[0]:
    print("\nChecking 'audio' field:")
    print(f"First audio path: {dataset[0]['audio']}")

if 'task' in dataset[0]:
    print("\nTask distribution:")
    tasks = [item['task'] for item in dataset]
    print({task: tasks.count(task) for task in set(tasks)})

if 'updrs' in dataset[0]:
    print("\nUPDRS distribution (if available):")
    updrs_levels = [item['updrs'] for item in dataset if 'updrs' in item]
    print({level: updrs_levels.count(level) for level in set(updrs_levels)})
